In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [109]:
texts = pd.read_csv('scipop_test2.csv', encoding = 'utf-8') #пока наилучший вариант был с третьей итерацией

In [110]:
texts = texts.drop(['Unnamed: 0'], axis=1)

Танцы с бубном для лемматизированных текстов

In [111]:
commons = open('common_lemmas.txt', 'r', encoding = 'utf-8')
commons = list(map(lambda x:x.strip(), commons))

In [112]:
talks = open('talks.txt', 'r', encoding = 'utf-8')
talks = list(map(lambda x:x.strip(), talks))

In [113]:
texts[:10]

,text,category
0,Исследование ученых из Американского онкологич...,1
1,Палеонтологи из США и Канады описали новый вид...,1
2,Исследователи из Медицинской школы Университет...,1
3,Некоторые самцы краба-скрипача нестараются выр...,1
4,"Ученые из Норвегии, Германии, Исландии, Швеции...",1
5,Археологи нашли в Испании свидетельства канниб...,1
6,"Трудовой коллектив, состоящий из друзей, работ...",1
7,Астрономы с помощью телескопа «Хаббл» впервые ...,1
8,Ученые из биологической лаборатории острова Ма...,1
9,Физики Московского государственного университе...,1


In [114]:
import re
import pymorphy2 as pm
from stop_words import get_stop_words

In [115]:
morph = pm.MorphAnalyzer()
def words(text):
    nolinks = ' '.join([word for word in text.split() if (not re.findall('https?://|\w\.\w', word))])#удалили ссылки
    clean_line = re.sub('[\W\d_-]+', ' ', nolinks.lower().strip())
    ws = re.split(' +', clean_line)
    return [morph.parse(w)[0].normal_form for w in ws]

texts_clean = texts['text'] #прочитали тексты
texts_list = [' '.join(words(text)) for text in texts_clean] # список списков слов

In [116]:
cleaned = pd.DataFrame({'col':texts_list})
cleaned.columns = ['clean']

In [11]:
#cleaned.to_csv('scipop9_clean.csv', encoding='utf-8')

In [117]:
def common_compare(text): #сраниваем текст со списком общих слов
    text_len = len(text.split())
    common_words = 0
    for word in text.split():
        if word in commons:
            common_words += 1
    return round(common_words/text_len, 2) 

In [118]:
def talks_compare(text): #сраниваем текст со списком общих слов
    text_len = len(text.split())
    talk_words = 0
    for word in text.split():
        if word in talks:
            talk_words += 1
    return round(talk_words/text_len, 2) 

**А здесь начинаем уже машинное обучение**

In [119]:
cleaned = cleaned.join(texts, lsuffix='cleaned', rsuffix='texts')

In [120]:
cleaned[:10]

,clean,text,category
0,исследование учёный из американский онкологиче...,Исследование ученых из Американского онкологич...,1
1,палеонтолог из сша и канада описать новый вид ...,Палеонтологи из США и Канады описали новый вид...,1
2,исследователь из медицинский школа университет...,Исследователи из Медицинской школы Университет...,1
3,некоторый самец краб скрипач нестараться вырыт...,Некоторые самцы краба-скрипача нестараются выр...,1
4,учёный из норвегия германия исландия швеция и ...,"Ученые из Норвегии, Германии, Исландии, Швеции...",1
5,археолог найти в испания свидетельство канниба...,Археологи нашли в Испании свидетельства канниб...,1
6,трудовой коллектив состоять из друг работать э...,"Трудовой коллектив, состоящий из друзей, работ...",1
7,астроном с помощь телескоп хаббл впервые зафик...,Астрономы с помощью телескопа «Хаббл» впервые ...,1
8,учёный из биологический лаборатория остров мау...,Ученые из биологической лаборатории острова Ма...,1
9,физика московский государственный университет ...,Физики Московского государственного университе...,1


In [121]:
cleaned.to_csv('scipop9_clean.csv', encoding='utf-8')

In [122]:
train, test = train_test_split(cleaned, test_size=0.3)

In [123]:
y_train = train['category'] 
y_test = test['category']

In [124]:
train

,clean,text,category
86,мы то есть n продолжать наш проект сделать сам...,"Мы, то есть N + 1, продолжаем наш проект «Сдел...",1
24,аргентинский учёный провести ревизия останки п...,Аргентинские ученые провели «ревизию» останков...,1
146,говорить о индекс цитирование я бы разделить п...,"Говоря об индексах цитирования, я бы разделил...",1
337,психология с давний пора пытаться ответить на ...,Психология с давних пор пыталась ответить на в...,1
137,хорошо сохраниться мозг быть извлечь из череп ...,Хорошо сохранившийся мозг был извл...,1
154,в весь мир принять система грант когда професс...,Во всем мире принята система грантов. Когда ...,1
364,исследовательский центр ibm совместно с универ...,Исследовательский центр IBM совместно с универ...,2
214,wow сигнал один из самый известный радиосигнал...,Wow-сигнал — один из самых известных радиосигн...,1
87,контроллер повсюду этот небольшой электронный ...,Контроллеры повсюду. Эти небольшие электронные...,1
151,никакой радикальный изменение функция полиция ...,Никаких радикальных изменений функций полици...,1


In [125]:
import ksslib.readability_metrics as kk

In [126]:
def percent_syll(text):
    perc_diff = float(float(kk.diffsyll(text)))/float(len(kk.sentence_splitter(text)))
    return round(perc_diff, 2) 

In [127]:
def statist_vectors(text, clean):
    FRE = kk.flesch_RE(text)
    #FKG = kk.flesch_kincaid_grade(text)
    #SMOG = kk.smog_index(text)
    CLI = kk.coleman_liau_index(text)
    DCH = kk.dale_chall_score(text)
    GF = kk.gunning_fog(text)
    ASPS = kk.avg_syllab_per_sent(text)#дополнительно
    ASLC = kk.avg_letter_per_sent(text)
    #AWL = kk.avg_letter_per_word(text)
    CW = common_compare(clean)
    TW = talks_compare(clean)
    DW = percent_syll(text)
    return [FRE, CLI, DCH, GF, ASPS, ASLC, CW, TW, DW]

In [33]:
#try_me = statist_vectors(train['text'][1])

In [34]:
#try_me

In [128]:
def count_me_all(texts, cleaned):
    return [statist_vectors(texts.values[i], cleaned.values[i]) for i in range(len(texts))]

In [129]:
x_train = count_me_all(train['text'], train['clean'])
x_test = count_me_all(test['text'], train['clean'])

In [48]:
#test['text'].values[0]

In [24]:
from sklearn.preprocessing import StandardScaler

In [130]:
scaler = StandardScaler()
x_fit = scaler.fit(x_train)
x_transform = scaler.transform(x_train)
x_fit_test = scaler.fit(x_test)
x_transform_test = scaler.transform(x_test)

In [131]:
x_transform

array([[ 1.5443478 , -0.85445075, -1.41974519, ...,  0.49467385,
         1.41569839, -1.44284631],
       [ 0.94821218, -0.76298971, -1.09307611, ...,  0.08968853,
         1.64073047, -0.98762754],
       [ 0.82638424, -0.74469751, -0.57491135, ...,  1.43963959,
         1.19066631, -0.48934754],
       ...,
       [-0.90658404,  0.93513693,  1.61039217, ..., -1.53025274,
        -1.9597828 ,  0.96858285],
       [ 0.90714659, -0.48250919, -0.0792755 , ...,  1.03465427,
         0.06550591, -0.85229223],
       [-0.92164143,  1.62719214,  1.15981412, ..., -1.53025274,
        -1.28468656,  0.62409297]])

In [27]:
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [132]:
model_svc = SVC()
model_svc.fit(x_transform, y_train)
print(model_svc)

expected_svc = y_test
predicted_svc = model_svc.predict(x_transform_test)

print(metrics.classification_report(expected_svc, predicted_svc))
print(metrics.confusion_matrix(expected_svc, predicted_svc))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

          1       0.77      1.00      0.87        97
          2       0.00      0.00      0.00        29

avg / total       0.59      0.77      0.67       126

[[97  0]
 [29  0]]


C:\Users\Ksenia\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [135]:
model = KNeighborsClassifier(n_neighbors=3, algorithm='ball_tree', leaf_size = 100, p=10, n_jobs = 7)
model.fit(x_transform, y_train)
print(model)

expected = y_test
predicted = model.predict(x_transform_test)

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))


KNeighborsClassifier(algorithm='ball_tree', leaf_size=100, metric='minkowski',
           metric_params=None, n_jobs=7, n_neighbors=3, p=10,
           weights='uniform')
             precision    recall  f1-score   support

          1       0.78      0.97      0.87        97
          2       0.50      0.10      0.17        29

avg / total       0.72      0.77      0.71       126

[[94  3]
 [26  3]]


In [52]:
from sklearn.svm import LinearSVC
clf = LinearSVC().fit(x_transform, y_train)
predicted = clf.predict(x_transform_test)

In [53]:
acc = accuracy_score(predicted, y_test)
micro_f1 = f1_score(predicted, y_test, average = 'micro')
micro_p = precision_score(predicted, y_test, average = 'micro')
micro_r = recall_score(predicted, y_test, average = 'micro')
macro_f1 = f1_score(predicted, y_test, average = 'macro')
macro_p = precision_score(predicted, y_test, average = 'macro')
macro_r = recall_score(predicted, y_test, average = 'macro')
print('acc={0:1.4f}'.format(acc))
print('micro F1={0:1.4f}, micro P={1:1.4f}, micro R={2:1.4f}'.format(micro_f1, micro_p, micro_r))
print('macro F1={0:1.4f}, macro P={1:1.4f}, macro R={2:1.4f}\n'.format(macro_f1, macro_p, macro_r))

acc=0.5079
micro F1=0.5079, micro P=0.5079, micro R=0.5079
macro F1=0.3613, macro P=0.4087, macro R=0.3246



C:\Users\Ksenia\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Ksenia\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Дерево решений

In [54]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(x_transform, y_train)
predicted = clf.predict(x_transform_test)

In [55]:
acc = accuracy_score(predicted, y_test)
micro_f1 = f1_score(predicted, y_test, average = 'micro')
micro_p = precision_score(predicted, y_test, average = 'micro')
micro_r = recall_score(predicted, y_test, average = 'micro')
macro_f1 = f1_score(predicted, y_test, average = 'macro')
macro_p = precision_score(predicted, y_test, average = 'macro')
macro_r = recall_score(predicted, y_test, average = 'macro')
print('acc={0:1.4f}'.format(acc))
print('micro F1={0:1.4f}, micro P={1:1.4f}, micro R={2:1.4f}'.format(micro_f1, micro_p, micro_r))
print('macro F1={0:1.4f}, macro P={1:1.4f}, macro R={2:1.4f}\n'.format(macro_f1, macro_p, macro_r))

acc=0.3889
micro F1=0.3889, micro P=0.3889, micro R=0.3889
macro F1=0.3346, macro P=0.3405, macro R=0.3403

